<a href="https://colab.research.google.com/github/rkaclfdl123/TIL/blob/master/train_yolov5_pistols_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 9959, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 9959 (delta 13), reused 16 (delta 9), pack-reused 9928
Receiving objects: 100% (9959/9959), 10.32 MiB | 25.05 MiB/s, done.
Resolving deltas: 100% (6895/6895), done.


In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Xp2OlmobAQMlFGrRu9rO")
project = rf.workspace().project("knife-fwnwz")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to knife-1 in yolov5pytorch:: 100%|██████████| 2456/2456 [00:01<00:00, 1522.50it/s]


In [4]:
%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5


In [4]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images

nc: 1
names: ['pistol']

In [5]:
%cd /
from glob import glob

img_list = glob('/content/knife-1/train/images/*.jpg')

print(len(img_list))

/
1225


In [6]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

980 245


In [7]:
train_img_list

['/content/knife-1/train/images/H_8211-91-0000_31_050_png.rf.f0b5aa7bb54645c3d14a213616598c48.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_18_547_png.rf.0d589660531f41544b895805304ad020.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_15_581_png.rf.2bc0a06e901fd1c95b9a02f382518f2a.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_32_050_png.rf.07ce02a10046a28180e22396747077f3.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_29_054_png.rf.c8c68709da7456b789f7a6d847faed05.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_15_572_png.rf.224c04431fc0ad6f18e58a209cfe4eb0.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_18_564_png.rf.cb772edd9cde1e4cbac98d7db96c489e.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_18_551_png.rf.0a9876496fa8f3cd84883312aa2dd2bd.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_31_048_png.rf.24a057b7e88b68012fe54339cb8879c1.jpg',
 '/content/knife-1/train/images/H_8211-91-0000_01_551_png.rf.539d57d7a33aeef87ba8a37a22d5eed9.jpg',


In [8]:
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [9]:
import yaml

with open('/content/knife-1/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/val.txt'

with open('/content/knife-1/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['Knife'], 'nc': 1, 'train': 'knife-1/train/images', 'val': 'knife-1/valid/images'}
{'names': ['Knife'], 'nc': 1, 'train': '/content/dataset/train.txt', 'val': '/content/dataset/val.txt'}


In [12]:
%cd /content/dataset/yolov5/

!python train.py --img 416 --batch 16 --epochs 50 --data /content/knife-1/data.yaml --cfg ./models/yolov5s.yaml --weights yolov5s.pt --name gun_yolov5s_results

/content/dataset/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/knife-1/data.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=gun_yolov5s_results, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-92-ge80a09b torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gam

In [15]:
from IPython.display import Image
import os

val_img_path = val_img_list[4]

!python detect.py --weights /content/dataset/yolov5/runs/train/gun_yolov5s_results/weights/best.pt --img 416 --conf 0.5 --source "{val_img_path}"

Image(os.path.join('/content/yolov5/inference/output', os.path.basename(val_img_path)))

detect: weights=['/content/dataset/yolov5/runs/train/gun_yolov5s_results/weights/best.pt'], source=/content/knife-1/train/images/H_8211-91-0000_02_576_png.rf.face7a4564ebf1d60cde70741b1bd168.jpg, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-92-ge80a09b torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /content/knife-1/train/images/H_8211-91-0000_02_576_png.rf.face7a4564ebf1d60cde70741b1bd168.jpg: 416x416 1 Knife, Done. (0.006s)
Speed: 0.3ms pre-process, 6.4ms inference, 1.3ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp2
